In [ ]:
from flask import Flask, json, Response, request
import pickle
import requests
from threading import Thread
import pandas as pd
import json

In [ ]:
from google.colab import drive

In [ ]:
from google.colab import auth
auth.authenticate_service_account()

Successfully saved credentials for pipelineauth@weatherlink-404323.iam.gserviceaccount.com


In [ ]:
from google.cloud import bigquery
project_name='weatherlink-404323'
client = bigquery.Client(project=project_name)
dataset_name = 'weatherlink_master'

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
app = Flask(__name__)

In [ ]:
model = pickle.load(open("/content/drive/MyDrive/decision_tree_0_74.sav", 'rb'))

In [ ]:
le = LabelEncoder()

In [ ]:
# Run a python(flask)-based web service in your note book
# You can reload this cell to restart the server if you make changes

default_port = 6060

from werkzeug.serving import make_server
from flask import Flask
import threading

class ServerThread(threading.Thread):

    def __init__(self, app, port):
        threading.Thread.__init__(self)
        self.port = port
        self.srv = make_server('127.0.0.1', port, app)
        self.ctx = app.app_context()
        self.ctx.push()

    def run(self):
        print('starting server on port:',self.port)
        self.srv.serve_forever()

    def shutdown(self):
        self.srv.shutdown()

def start_server(port=default_port):
    global server
    if 'server' in globals() and server:
        print('stopping server')
        stop_server()

    app = Flask('myapp')


    @app.route("/predict", methods = ['POST'])
    def predict_crashes():
        input_data = request.json
        df = pd.DataFrame.from_dict(json.loads(input_data))
        df["day_of_week_name_le"] = le.fit_transform(df["day_of_week_name"])
        df["atmospheric_cond_name_le"] = le.fit_transform(df["atmospheric_cond_name"])

        df_temp = df[['week_of_crash', 'day_of_week_name_le', 'atmospheric_cond_name_le', 'no_car_ratio', 'male_pop_ratio', 'median_age', 'median_rent', 'income_per_capita' ]]

        predictions = model.predict(df_temp)
        df['predictions'] = predictions
        return df[['geoid', 'predictions']].to_json()

    server = ServerThread(app,port)
    server.start()

def stop_server():
    global server
    if server:
        server.shutdown()
        server = None

# Start the server here
start_server()

stopping server
starting server on port: 6060


In [ ]:
p = requests.get('http://localhost:6060/predict')

INFO:werkzeug:127.0.0.1 - - [13/Dec/2023 03:10:18] "GET /predict HTTP/1.1" 405 -


In [ ]:
x = requests.post('http://localhost:6060/predict', json = df[0:10].to_json())

INFO:werkzeug:127.0.0.1 - - [13/Dec/2023 03:26:39] "POST /predict HTTP/1.1" 200 -


In [ ]:
x.text

'{"geoid":{"0":12033,"1":12033,"2":12033,"3":12033,"4":12033,"5":12033,"6":12033,"7":12033,"8":12033,"9":12033},"predictions":{"0":1.0,"1":3.0,"2":1.0,"3":1.0,"4":3.0,"5":1.0,"6":1.0,"7":1.0,"8":2.0,"9":3.0}}'

In [ ]:
pd.DataFrame.from_dict(json.loads(x.text))

,geoid,predictions
0,12033,1.0
1,12033,3.0
2,12033,1.0
3,12033,1.0
4,12033,3.0
5,12033,1.0
6,12033,1.0
7,12033,1.0
8,12033,2.0
9,12033,3.0


In [ ]:
sql_query = """

select
*
FROM
`weatherlink_master.census_accident_master_filtered`

"""

df = client.query(sql_query).to_dataframe()
print(df)

        geoid timestamp_of_crash  year  month_of_crash  week_of_crash  \
0       12033         2016-11-19  2016              11             46   
1       12033         2016-05-29  2016               5             22   
2       12033         2016-05-06  2016               5             18   
3       12033         2016-10-07  2016              10             40   
4       12033         2016-03-20  2016               3             12   
...       ...                ...   ...             ...            ...   
115647  55039         2019-09-01  2019               9             35   
115648  55039         2019-06-06  2019               6             22   
115649  55039         2019-01-13  2019               1              2   
115650  55039         2019-12-05  2019              12             48   
115651  55039         2019-06-02  2019               6             22   

       day_of_week_name  hour_of_crash atmospheric_cond_name  \
0              Saturday              0                 Clea

In [ ]:
df[0:10].to_json()

'{"geoid":{"0":12033,"1":12033,"2":12033,"3":12033,"4":12033,"5":12033,"6":12033,"7":12033,"8":12033,"9":12033},"timestamp_of_crash":{"0":"2016-11-19","1":"2016-05-29","2":"2016-05-06","3":"2016-10-07","4":"2016-03-20","5":"2016-02-27","6":"2016-05-19","7":"2016-09-03","8":"2016-02-17","9":"2016-04-27"},"year":{"0":2016,"1":2016,"2":2016,"3":2016,"4":2016,"5":2016,"6":2016,"7":2016,"8":2016,"9":2016},"month_of_crash":{"0":11,"1":5,"2":5,"3":10,"4":3,"5":2,"6":5,"7":9,"8":2,"9":4},"week_of_crash":{"0":46,"1":22,"2":18,"3":40,"4":12,"5":8,"6":20,"7":35,"8":7,"9":17},"day_of_week_name":{"0":"Saturday","1":"Sunday","2":"Friday","3":"Friday","4":"Sunday","5":"Saturday","6":"Thursday","7":"Saturday","8":"Wednesday","9":"Wednesday"},"hour_of_crash":{"0":0,"1":23,"2":4,"3":23,"4":1,"5":15,"6":22,"7":22,"8":21,"9":19},"atmospheric_cond_name":{"0":"Clear","1":"Clear","2":"Clear","3":"Clear","4":"Cloudy","5":"Clear","6":"Cloudy","7":"Cloudy","8":"Clear","9":"Cloudy"},"manner_of_collision_name":{"